In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import mean_squared_error
import random

# Load dataset
data = pd.read_csv('sp500_fred.csv')

# Preprocess the data (same as before)
data['DATE'] = pd.to_datetime(data['DATE'])
data.set_index('DATE', inplace=True)
data.dropna(subset=['SP500'], inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['SP500']])

# Create sequences of data for CNN
seq_length = 60
X = []
y = []
for i in range(seq_length, len(data_scaled)):
    X.append(data_scaled[i-seq_length:i, 0])  # Features (SP500)
    y.append(data_scaled[i, 0])  # Target (SP500)

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Reshape for CNN (1D)

# Split data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Objective Function (to be optimized by SWO)
def objective(params):
    units = params[0]  # Number of filters (CNN units)
    dropout_rate = params[1]  # Dropout rate

    model = Sequential()
    model.add(Conv1D(filters=units, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=1))  # Predict a single value (SP500 index)
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Spider Wasp Optimization (SWO) Implementation
def swarm_optimization(num_wasps=10, iterations=20):
    # Initialize the swarm (wasps)
    swarm = []
    for _ in range(num_wasps):
        units = random.randint(50, 200)  # Randomly initialize number of filters (CNN units)
        dropout_rate = random.uniform(0.1, 0.5)  # Randomly initialize dropout rate
        swarm.append([units, dropout_rate])

    best_position = None
    best_value = float('inf')
    
    # Optimization loop
    for _ in range(iterations):
        for i, wasp in enumerate(swarm):
            value = objective(wasp)
            if value < best_value:
                best_value = value
                best_position = wasp

        # Update positions based on exploration/exploitation rules (simplified)
        for i in range(num_wasps):
            if random.random() < 0.5:
                swarm[i] = [best_position[0] + random.randint(-5, 5), 
                            best_position[1] + random.uniform(-0.05, 0.05)]
    
    return best_position

# Run SWO
best_params = swarm_optimization(num_wasps=10, iterations=20)
print(f"Best parameters: {best_params}")

# Build the final model using the best parameters found
best_units = best_params[0]
best_dropout_rate = best_params[1]

final_model = Sequential()
final_model.add(Conv1D(filters=best_units, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
final_model.add(MaxPooling1D(pool_size=2))
final_model.add(Flatten())
final_model.add(Dense(units=1))

final_model.compile(optimizer='adam', loss='mean_squared_error')
final_model.fit(X_train, y_train, epochs=20, batch_size=32)

# Evaluate the model
y_pred = final_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Inverse transform to get actual S&P 500 values
y_pred_actual = scaler.inverse_transform(np.column_stack((X_test[:, -1, 0], y_pred)))[:, 1]
y_test_actual = scaler.inverse_transform(np.column_stack((X_test[:, -1, 0], y_test)))[:, 1]

# Plot the results
plt.plot(y_test_actual, color='blue', label='Actual S&P 500')
plt.plot(y_pred_actual, color='red', label='Predicted S&P 500')
plt.title('S&P 500 Prediction with CNN (SWO)')
plt.xlabel('Time')
plt.ylabel('S&P 500 Index')
plt.legend()
plt.show()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject